In [107]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


##load data



In [108]:
df=pd.read_csv("/content/Coursera.csv")

##EDA

In [109]:
print(df.info())
print(df.describe())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3522 entries, 0 to 3521
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Course Name         3522 non-null   object
 1   University          3522 non-null   object
 2   Difficulty Level    3522 non-null   object
 3   Course Rating       3522 non-null   object
 4   Course URL          3522 non-null   object
 5   Course Description  3522 non-null   object
 6   Skills              3522 non-null   object
dtypes: object(7)
memory usage: 192.7+ KB
None
                                              Course Name  \
count                                                3522   
unique                                               3416   
top     Google Cloud Platform Fundamentals: Core Infra...   
freq                                                    8   

                      University Difficulty Level Course Rating  \
count                       3522          

,Course Name,University,Difficulty Level,Course Rating,Course URL,Course Description,Skills
0,Write A Feature Length Screenplay For Film Or ...,Michigan State University,Beginner,4.8,https://www.coursera.org/learn/write-a-feature...,Write a Full Length Feature Film Script In th...,Drama Comedy peering screenwriting film D...
1,Business Strategy: Business Model Canvas Analy...,Coursera Project Network,Beginner,4.8,https://www.coursera.org/learn/canvas-analysis...,"By the end of this guided project, you will be...",Finance business plan persona (user experien...
2,Silicon Thin Film Solar Cells,�cole Polytechnique,Advanced,4.1,https://www.coursera.org/learn/silicon-thin-fi...,This course consists of a general presentation...,chemistry physics Solar Energy film lambda...
3,Finance for Managers,IESE Business School,Intermediate,4.8,https://www.coursera.org/learn/operational-fin...,"When it comes to numbers, there is always more...",accounts receivable dupont analysis analysis...
4,Retrieve Data using Single-Table SQL Queries,Coursera Project Network,Beginner,4.6,https://www.coursera.org/learn/single-table-sq...,In this course you�ll learn how to effectively...,Data Analysis select (sql) database manageme...


In [110]:

print(df.columns)
df.shape


Index(['Course Name', 'University', 'Difficulty Level', 'Course Rating',
       'Course URL', 'Course Description', 'Skills'],
      dtype='object')


(3522, 7)

##data cleaning

In [111]:
df.isnull().sum()

,0
Course Name,0
University,0
Difficulty Level,0
Course Rating,0
Course URL,0
Course Description,0
Skills,0


In [112]:

for col in df.columns:
  values_in_col=df[col].unique()
  print(col,values_in_col)

Course Name ['Write A Feature Length Screenplay For Film Or Television'
 'Business Strategy: Business Model Canvas Analysis with Miro'
 'Silicon Thin Film Solar Cells' ...
 'Business intelligence and data analytics: Generate insights'
 'Rigid Body Dynamics'
 'Architecting with Google Kubernetes Engine: Production']
University ['Michigan State University' 'Coursera Project Network'
 '�cole Polytechnique' 'IESE Business School'
 'The Chinese University of Hong Kong' 'University of Washington'
 'The State University of New York' 'Saint Petersburg State University'
 'University of California, Irvine' 'University of Colorado System'
 'Rice University' 'University of Geneva'
 'University of California, Davis'
 '�cole Polytechnique F�d�rale de Lausanne' 'Google Cloud'
 'National Research Tomsk State University' 'University of Florida'
 'Johns Hopkins University' 'Universiteit Leiden'
 'The University of Edinburgh' 'Columbia University'
 'Korea Advanced Institute of Science and Technology(KAIS

In [113]:
df['Course Rating']=df['Course Rating'].replace({'Not Calibrated':np.nan})
df.isnull().sum()

,0
Course Name,0
University,0
Difficulty Level,0
Course Rating,82
Course URL,0
Course Description,0
Skills,0


In [114]:
df['Course Rating']=pd.to_numeric(df['Course Rating'])

In [115]:
df['Course Rating']=df['Course Rating'].fillna(value=df['Course Rating'].mean())

In [116]:

df.isnull().sum()

,0
Course Name,0
University,0
Difficulty Level,0
Course Rating,0
Course URL,0
Course Description,0
Skills,0


In [117]:
df.duplicated().sum()

np.int64(98)

In [118]:
df=df.drop_duplicates()
df.duplicated().sum()

np.int64(0)

##pre processing

In [119]:
#extract featurs
df['text_document']=df['Course Name'] + " " + df['University'] + " " + df['Skills'] + " " + df['Course Description']


/tmp/ipython-input-119-2539663505.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text_document']=df['Course Name'] + " " + df['University'] + " " + df['Skills'] + " " + df['Course Description']


In [120]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [121]:
port_stem=PorterStemmer()
def stemming(text_document):
  stemmed_text=re.sub('[^a-zA-Z]',' ',text_document)
  stemmed_text=stemmed_text.lower().split()
  stemmed_text=[port_stem.stem(word) for word in stemmed_text if not word in stopwords.words('english') ]
  stemmed_text=' '.join(stemmed_text)
  return stemmed_text

In [122]:
df['text_document']=df['text_document'].apply(stemming)
df['text_document'].values

array(['write featur length screenplay film televis michigan state univers drama comedi peer screenwrit film document review dialogu creativ write write unix shell art human music art write full length featur film script cours write complet featur length screenplay film televis seriou drama romant comedi anyth learn break creativ process compon discov structur process allow produc polish pitch readi script end cours complet project increas confid idea abil feel prepar pitch first script get start next cours design tap creativ base activ learn actual learn take place within activ write learn link trailer cours view trailer pleas copi past link browser http vimeo com b b dc learner review love approach professor wheeler take toward cours point easi follow inform would definit recommend anyon interest take screenplay write cours cours curriculum simpl adopt profession writer room process write post work peer review share feedback peer revis work feedback receiv peer real world feel profes

In [124]:
tfidf=TfidfVectorizer()
content=df['text_document'].values
tfidf_matrix=tfidf.fit_transform(content)

In [125]:
user_input=input("enter your enterests: ")
user_input_vector=tfidf.transform([user_input])
similarity_scores=cosine_similarity(user_input_vector,tfidf_matrix)



enter your enterests: AI data analysis nlp 


In [134]:
top_indices = np.argsort(similarity_scores, axis=1)[:, -5:].flatten()[::-1]
top_courses=df.iloc[top_indices]
top_courses

,Course Name,University,Difficulty Level,Course Rating,Course URL,Course Description,Skills,text_document
779,Introduction to Natural Language Processing in...,Coursera Project Network,Beginner,3.5,https://www.coursera.org/learn/intro-natural-l...,"In this 1-hour long project-based course, you ...",Language Learning Python Programming Experim...,introduct natur languag process python courser...
2175,AI For Everyone,DeepLearning.AI,Conversant,4.8,https://www.coursera.org/learn/ai-for-everyone,AI is not only for engineers. If you want your...,Deep Learning Strategy project plan Human L...,ai everyon deeplearn ai deep learn strategi pr...
2573,Introduction to Artificial Intelligence (AI),IBM,Conversant,4.7,https://www.coursera.org/learn/introduction-to-ai,In this course you will learn what Artificial ...,Machine Learning artificial general intellige...,introduct artifici intellig ai ibm machin lear...
1361,Deep Learning NLP: Training GPT-2 from scratch,Coursera Project Network,Beginner,4.1,https://www.coursera.org/learn/deep-learning-n...,"In this 1-hour long project-based course, we w...",Natural Language Processing iOS Development ...,deep learn nlp train gpt scratch coursera proj...
1457,Natural Language Processing with Attention Models,DeepLearning.AI,Beginner,3.4,https://www.coursera.org/learn/attention-model...,In Course 4 of the Natural Language Processing...,speech synthesis language natural language ...,natur languag process attent model deeplearn a...


In [129]:
import joblib
joblib.dump(tfidf, 'tfidf_vectorizer.pkl')
joblib.dump(tfidf_matrix, 'tfidf_matrix.pkl')
df.to_csv("processed_data.csv", index=False)
